In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import os
import pandas as pd
from datetime import datetime
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import chromedriver_autoinstaller as chromedriver
chromedriver.install()

#폴더 내 어플 링크 txt파일 읽어오는 함수
def alexcelread() :
    path = "./" #현재 디렉토리
    file_list = os.listdir(path)
    keyword_list_excel = [file for file in file_list if file.endswith("키워드.xlsx")] #키워드.xlsx로 끝나는 파일 찾기
    return keyword_list_excel #파일 리스트 반환

def Get_Pages(x) :
    webdriver_options = webdriver.ChromeOptions()
    webdriver_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36") #크롤링 감지 방지를 위한 user-agent 수정
    webdriver_options.add_argument('headless') #크롬 브라우저 띄우지 않는 옵션. 크롤링 화면을 보고 싶다면 #을 이용해 주석처리 할 것
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    
    url = "https://www.google.com/search?q=" + x #키워드인 x를 받아 해당 키워드에 대한 검색결과 URL을 생성함
    
    driver.get(url) #화면 띄우기
    
    page_to_return = [] #검색결과 페이지 링크를 저장할 리스트
    
    pages = driver.find_elements("xpath" , "//tr[@jsname='TeSSVd']/td/a") #검색결과 페이지 링크
    
    page_to_return.append(url) #pages를 통해 불러온 링크는 1페이지 링크를 포함하지 않음. 1페이지 링크 추가
    
    for i in range(len(pages)) :
        page_to_return.append(pages[i].get_attribute('href')) #나머지 검색페이지 링크 추가
    
    return page_to_return #검색페이지 10개 반환

#크롤링을 진행하는 함수

def crawlerX(x) : 
    webdriver_options = webdriver.ChromeOptions()
    webdriver_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36") #크롤링 감지 방지를 위한 user-agent 수정
    webdriver_options.add_argument('headless') #크롬 브라우저 띄우지 않는 옵션. 크롤링 화면을 보고 싶다면 #을 이용해 주석처리 할 것
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    
    driver.get(x) #화면 띄우기
    
    total = [] #Name과 URL을 합칠 변수 선언
    
    URL = driver.find_elements("xpath" ,"//div[@class='yuRUbf']/a") #링크 불러오기
    Name = driver.find_elements("xpath" ,"//div[@class='yuRUbf']/a/h3") #링크 별 제목 불러오기
    
    for i in range(len(Name)) :
        total.append({ 'Name' : str(Name[i].text),
                 'URL' : str(URL[i].get_attribute('href')) })

    driver.quit() #웹드라이버 종료 및 크롬 창 종료. 메모리 관리 목적.
    
    return pd.DataFrame(total) #크롤링된 데이터 반환

#전체 함수를 조합한 함수(실행해야 하는 함수)
def web_crawler() :
    
    print("Start", datetime.today().strftime("%Y/%m/%d %H:%M:%S"))
    
    total_df = pd.DataFrame([]) #저장을 위한 데이터 프레임 우선 선언
    excel_list = alexcelread() #엑셀 파일 리스트 불러오기
    
    for a in excel_list : #엑셀 파일 리스트에서 하나씩 불러온다 = a
        
        print(a + " is opened", datetime.today().strftime("%Y/%m/%d %H:%M:%S"))
        
        excel = pd.read_excel(a) #a라는 엑셀 파일을 열어 excel로 선언한다
        keyword_list = excel["Keyword"] #excel에서 키워드를 불러와 keyword_list에 저장한다
        
        for i in keyword_list : #keyword_list에서 키워드를 하나씩 불러온다 = i
            
            print(str(i) + " keyword crawling is started", datetime.today().strftime("%Y/%m/%d %H:%M:%S"))
            
            for x in Get_Pages(i) : #Get_Pages에 키워드(i)를 넣는다. Get_Pages에서는 키워드에 대한 검색결과 링크를 리스트로 반환한다. 검색결과 링크 리스트를 하나씩 불러온다 = x
                total_df = pd.concat([total_df, crawlerX(x)]) #crawlerX 함수에 검색결과 링크(x)를 입력, 크롤링 데이터가 포함된 딕셔너리를 받아온다. 이후 total_df와 합친다(10개 검색 페이지 결과를 합치기 위함)
                time.sleep(5) #다음 검색결과 페이지로 진행하기 전 5초를 기다린다(리소스 관리목적, 사용 환경에 따라 조절 가능)
            total_df.reset_index(drop=True, inplace = True) #인덱스를 초기화 한다
            
            if not os.path.exists(a[:-5] + "크롤링데이터.xlsx") : #엑셀 파일 이름은 엑셀 파일 이름에 크롤링데이터를 붙인 방식
                with pd.ExcelWriter(a[:-5] + "크롤링데이터.xlsx", mode='w', engine='openpyxl') as writer : #엑셀 파일 없으면 생성
                    total_df.to_excel(writer, sheet_name = str(i)) #시트 이름은 각 키워드 이름
            else: 
                with pd.ExcelWriter(a[:-5] + "크롤링데이터.xlsx", mode='a', engine='openpyxl') as writer : #엑셀 파일 있으면 해당 파일 내 시트 추가
                    total_df.to_excel(writer, sheet_name = str(i)) #시트 이름은 각 키워드 이름 이름
            
            total_df = pd.DataFrame([]) #딕셔너리를 초기화해 다른 키워드의 크롤링을 준비한다.
                
    print("DONE", datetime.today().strftime("%Y/%m/%d %H:%M:%S"))
    
web_crawler()

In [3]:
Mbti_data = pd.read_excel('C:/Users/dskim/Mbti crowling data.xlsx')
Mbti_data

,Unnamed: 0,Name,URL
0,0,INFJ - 나무위키,https://namu.wiki/w/INFJ
1,1,성격유형 : “옹호자” (INFJ) - 16Personalities,https://www.16personalities.com/ko/%EC%84%B1%E...
2,2,INFJ 유형 특징 팩폭 연애유형 총정리 - Naver Post - 네이버,https://post.naver.com/viewer/postView.nhn?vol...
3,3,"MBTI 성격 유형검사: INFJ(인프제) 유형의 특징, 연애, 직업, 궁합 ...",https://2trillionlee.tistory.com/entry/MBTI-%E...
4,4,[MBTI] 죽었다 깨어나도 나는 Infj - 브런치,https://brunch.co.kr/@findhyo/32
5,5,성격 유형 - 16Personalities,https://www.16personalities.com/ko/%EC%84%B1%E...
6,6,"INFJ 특징 6가지 정리 | INFJ 성격, 직업, 연애, 궁합, 팩폭, 연예인",https://ddnews.co.kr/infj-%ED%8A%B9%EC%A7%95/
7,7,All About the INFJ Personality Type | Truity,https://www.truity.com/personality-type/INFJ
8,8,"INFJ: Introverted, Intuitive, Feeling, Judging...",https://www.verywellmind.com/infj-introverted-...
9,9,인프제 INFJ의 그라운드 : 감탄 굿즈 마켓 oround,https://www.oround.com/selleb/infj/admin?prear...


In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import load_workbook


url = 'https://www.16personalities.com/ko/%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95-infj'
html = requests.get(url)
soup = BeautifulSoup(html.text)
content = soup.select_one('article').get_text()



        
word_list = content.lower().split()

def clean_wordlist(input_list):
    output_list = []
    for word in input_list:
        symbols = """!@#$%^&*()_-+={[}]|\;:"‘'·<>?/., """
        for i in range(len((symbols))):
            word = word.replace(symbols[i], '')      
        if len(word) > 0:
            output_list.append(word)
    return output_list
    
clean_list = clean_wordlist(word_list)

def counter(input_list):
    word_count = {}
    for word in clean_list:
        if word in  word_count:
            word_count[word] += 1
        else:
            word_count[word] = 1
    return word_count
    
word_count = counter(clean_list)
word_count
# >>> {'as': 6, 'long': 4, 'i': 4, ..., 'bless': 1}

word_count = sorted(word_count.items(), key=lambda x:x[1], reverse=True)
word_count

[('다른', 14),
 ('합니다', 9),
 ('이들은', 9),
 ('옹호자는', 9),
 ('자신의', 7),
 ('사람을', 5),
 ('위해', 5),
 ('있습니다', 5),
 ('깊은', 5),
 ('대신', 4),
 ('때가', 4),
 ('매우', 3),
 ('성격으로', 3),
 ('삶에', 3),
 ('일을', 3),
 ('또한', 3),
 ('관계를', 3),
 ('자신을', 3),
 ('돕는', 3),
 ('사람과의', 3),
 ('때는', 2),
 ('사람의', 2),
 ('잠재력을', 2),
 ('하며', 2),
 ('수', 2),
 ('희귀한', 2),
 ('큰', 2),
 ('이들에게', 2),
 ('있다고', 2),
 ('아닙니다', 2),
 ('사회의', 2),
 ('가치를', 2),
 ('통해', 2),
 ('찾기', 2),
 ('노력합니다', 2),
 ('깊이', 2),
 ('인생의', 2),
 ('지니고', 2),
 ('주변', 2),
 ('사람들과', 2),
 ('못할', 2),
 ('느낄', 2),
 ('더', 2),
 ('최선을', 2),
 ('다합니다', 2),
 ('일에', 2),
 ('한다는', 2),
 ('일에서', 2),
 ('느낍니다', 2),
 ('많은', 2),
 ('문제를', 2),
 ('그러나', 2),
 ('가끔은', 2),
 ('돌보는', 2),
 ('있으며', 2),
 ('사람과', 2),
 ('행복감을', 2),
 ('능력이', 2),
 ('사람이', 2),
 ('사명을', 2),
 ('성격유형', 1),
 ('“옹호자”', 1),
 ('대할', 1),
 ('그', 1),
 ('봐야', 1),
 ('사람들이', 1),
 ('발휘할', 1),
 ('있도록', 1),
 ('도와야', 1),
 ('한다', 1),
 ('요한', 1),
 ('볼프강', 1),
 ('폰', 1),
 ('괴테', 1),
 ('옹호자infj는', 1),
 ('성격임에도', 1),
 ('불구하고', 1),
 ('세상에',

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import  openpyxl  as  op
from openpyxl import load_workbook


Mbti_data = load_workbook('Mbti crowling data.xlsx')
Mbti_sheet = Mbti_data.active

for row1 in range(2,34):
    url = Mbti_sheet.cell(row=row1, column=3).value
    html = requests.get(url)
    soup = BeautifulSoup(html.text)
    
    tag_list = ["article","div","span","a","h","p"]
    content = None
    for i in tag_list:
        for k in range(len(tag_list)):
            while content is not None:
                content = soup.select_one(i[k]).get_text()
                
word_list = content.lower().split()
    

def clean_wordlist(input_list):
    output_list = []
    for word in input_list:
        symbols = """!@#$%^&*()_-+={[}]|\;:"‘'·<>?/., """
        for i in range(len((symbols))):
            word = word.replace(symbols[i], '')      
        if len(word) > 0:
            output_list.append(word)
    return output_list
    
clean_list = clean_wordlist(word_list)

def counter(input_list):
    word_count = {}
    for word in clean_list:
        if word in  word_count:
            word_count[word] += 1
        else:
            word_count[word] = 1
    return word_count
    
word_count = counter(clean_list)
word_count
# >>> {'as': 6, 'long': 4, 'i': 4, ..., 'bless': 1}

word_count = sorted(word_count.items(), key=lambda x:x[1], reverse=True)
word_count

AttributeError: 'NoneType' object has no attribute 'lower'